#Deploy custom model

In [0]:
%pip install --upgrade databricks-langchain langchain-community langchain databricks-sql-connector

In [0]:
from langchain.chains import RetrievalQA

In [0]:
from databricks_langchain import ChatDatabricks

chat_model = ChatDatabricks(
    endpoint="databricks-meta-llama-3-1-405b-instruct",
    temperature=0.1,
    max_tokens=4096,
)

In [0]:
from databricks_langchain import DatabricksVectorSearch

vector_store = DatabricksVectorSearch(index_name="workspace.llm_rag.databricks_vector_index",columns=["url","content"],)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
# response=retriever.invoke("provide step-by-step process to do disaster recovery in  Databricks")

In [0]:
import mlflow
mlflow.langchain.autolog()

In [0]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate




system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(chat_model, prompt)
retrievalbot = create_retrieval_chain(retriever, question_answer_chain)



In [0]:
prediction=retrievalbot.invoke({'input':"provide step-by-step process to do disaster recovery in  Databricks"})

#infer signature

In [0]:
retrievalbot

#Register with MLFLOW

In [0]:

import mlflow
from mlflow.models.resources import (
    DatabricksServingEndpoint,
    DatabricksVectorSearchIndex,
)
code_path = "/Workspace/Users/somanathsankaran@gmail.com/llm-101/langchain_agent.py"


# example using langchain
with mlflow.start_run():
  logged_agent_info = mlflow.langchain.log_model(
    lc_model=code_path,
    name="langchain_agent", # This string is used as the path inside the MLflow model where artifacts are stored
    resources=[
      DatabricksServingEndpoint(endpoint_name="databricks-meta-llama-3-1-405b-instruct"),
      DatabricksVectorSearchIndex(index_name="workspace.llm_rag.databricks_vector_index"),
    ],
    input_example={'input': 'hi'}
  )

print(f"MLflow Run: {logged_agent_info.run_id}")
print(f"Model URI: {logged_agent_info.model_uri}")

# To verify that the model has been logged correctly, load the agent and call `invoke`:


In [0]:
import mlflow
model_uri='models:/m-4d5778fc028e4d2191521d8beff19ad2'
model = mlflow.langchain.load_model(model_uri)


In [0]:
model.invoke({"input":"provide step-by-step process to do disaster recovery in  Databricks"})

#Register the agent to Unity Catalog

In [0]:

mlflow.set_registry_uri("databricks-uc")

catalog_name = "workspace"
schema_name = "llm_rag"
model_name = "langchain_agent"

model_name = catalog_name + "." + schema_name + "." + model_name
uc_model_info = mlflow.register_model(model_uri=model_uri, name=model_name)